In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from dateutil import parser as dparser
import urllib.parse
import timeit

import numpy as np
import pandas as pd
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy import sparse
from datasets import load_dataset, DatasetDict, load_metric

from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DataCollatorWithPadding, TrainingArguments
import torch
from transformers import pipeline, Trainer

from util.config import config
from util.pyRanker import BM25
from util.web_query_v2 import web_query
from util.ticker import Ticker
import ipywidgets as widgets

import dash
from dash import html
from dash import dcc
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
classifier = pipeline(task='sentiment-analysis',model=model,tokenizer=tokenizer)
max_tokens = int(tokenizer.model_max_length)

In [3]:
#LOADING IN CUSTOM DATASET TO FINE TUNE ACCURACY OF MODEL
#needs verification whether it fits/work with the current model as the labels for the 2 models (pos, neg) are different
#
dataset = load_dataset(
   'financial_phrasebank', 'sentences_66agree')

#modifying dataset to fit format of pre-trained model
custom_data = dataset['train']
custom_data = custom_data.filter(lambda example: example['label'] % 2 == 0)

#more modification
def add_prefix(example):
    if example['label'] == 2:
        example['label'] = example['label'] - 1
    return example

#mapping in changes to dataset - 0 is negative, 1 is positive
custom_data = custom_data.map(add_prefix)
#print(custom_data['label'])
        
#tokenizer for processing data
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True)

#mapping and splitting up custom data into training and testing
tokenized_datasets = custom_data.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

valid_set = DatasetDict({
    'train': tokenized_datasets['train'],
    'test': tokenized_datasets['test']})
#print(valid_set)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#error handling
columns_to_return = ['input_ids', 'label', 'attention_mask']
valid_set.set_format(type='torch', columns=columns_to_return)

#print(valid_set)
#train model with new data and report accuracy
training_args = TrainingArguments("test-trainer")
metric = load_metric("accuracy")

#accuracy metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#fine tuning the model with custom dataset
trainer = Trainer(
    model,
    training_args,
    train_dataset=valid_set['train'],
    eval_dataset=valid_set['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

#check in future to see if this is feasible and makes sense with the current pre-trained model

Reusing dataset financial_phrasebank (C:\Users\rahul\.cache\huggingface\datasets\financial_phrasebank\sentences_66agree\1.0.0\a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\rahul\.cache\huggingface\datasets\financial_phrasebank\sentences_66agree\1.0.0\a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0\cache-4e0666b22dba4755.arrow
Loading cached processed dataset at C:\Users\rahul\.cache\huggingface\datasets\financial_phrasebank\sentences_66agree\1.0.0\a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0\cache-019abd29eeade3b4.arrow
Loading cached processed dataset at C:\Users\rahul\.cache\huggingface\datasets\financial_phrasebank\sentences_66agree\1.0.0\a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0\cache-ec3e9a7a9cb89ff5.arrow


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence.
***** Running training *****
  Num examples = 1513
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 570


Saving model checkpoint to test-trainer\checkpoint-500
Configuration saved in test-trainer\checkpoint-500\config.json
Model weights saved in test-trainer\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-trainer\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-trainer\checkpoint-500\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=570, training_loss=0.09987983954580207, metrics={'train_runtime': 1042.0894, 'train_samples_per_second': 4.356, 'train_steps_per_second': 0.547, 'total_flos': 93947586714396.0, 'train_loss': 0.09987983954580207, 'epoch': 3.0})

In [4]:
#running through sub_docs and outputting sentiment list

def get_sentiments(docs):
    
    c = 0
    for x in docs.keys():
        c+=len(docs[x])
    
    pgres = widgets.IntProgress(value=0,min=0,max=c, step=1)
    display(pgres)
    
    sentiments = {}
    
    for x in docs.keys():
        scrs=[]
        for y in range(0, len(docs[x])):
            
            s = classifier(docs[x][y])
            scr = s[0]['score']
            if s[0]['label']=="NEGATIVE":
                scr=scr*-1
            scrs.append(scr)
            pgres.value+=1
            pgres.description=str(pgres.value)+":"+str(c)
        
        sentiments[x]=scrs
                
    return sentiments

In [5]:
class Corpus(object):
    
    #******************************************************************************
    #----------------------------------Method---------------------------------------
    #******************************************************************************
    
    #Run the web_query to produce a collection of text documents scraped from the web
    
    #Use the set_results() function to store the full results in the corpus for processing
    
    #Use the set_corpus() function to assign the documents scraped from the web to the corpus
    
    #Sub divide the documents into smaller sub_docs
    
    #Rank the documents based on relevance to the original query as well as any tags
    
    #Prune the sub_docs to produce a relevant set
    
    #
    
    #******************************************************************************
    #******************************************************************************
    
    def __init__(self):
        
        #typical corpus data
        self.documents = []
        self.vocabulary = []
        self.number_of_documents = 0
        self.vocabulary_size = 0
                
        #for web results
        self.query_results=None
        self.max_tokens=512
        self.failed = []
        
        #sub dividing documents
        self.tokenizer=None
        self.sub_docs=None
        self.sub_list=[]
        
        #queries
        self.prime_q=None
        self.expanded_q=None        
        
        #relevance scores
        self.document_scores=None
        self.subdoc_scores=None
        self.title_scores=None
        self.sub_list_scores=None
                
        #pruned data
        self.pruned_docs=[]
        self.pruned_subdocs={}
        
        #relevant
        self.relevant_set={}
        self.relevant_scores={}
        self.rel_list_scores=[]
        
        #sentiments
        self.sentiments=None
    
    
    #******************************************************************************
    #------------------------------Setting Corpus----------------------------------
    #******************************************************************************
    
    def set_results(self, df):
        #dataframe returned from webquery
        self.query_results=df
    
    def set_corpus(self, documents):
        self.documents = documents
                

    def build_corpus_from_file(self, file_path):

        f = open(file_path, 'r')
        docs = f.readlines()
        for d in docs:
            self.documents.append(d)
        self.number_of_documents = len(docs)

        
    def build_vocabulary(self, stopwords):

        v = set([])
        for x in self.documents:
            tmp = set(x.split())
            tmp = {x for x in tmp if x.lower() not in stopwords}
                        
            v.update(tmp)
        
        v = list(v)
        self.vocabulary = v
        self.vocabulary_size = len(v)
    
    
    #******************************************************************************
    #---------------------------------Queries--------------------------------------
    #******************************************************************************
    
    def set_ticker_data(self, ticker):
        name = re.sub('(Inc|Company|Corporation|Corp|Co|Incorporated|LLC|,|\.| )',"",str(ticker.name), flags=re.I)
        self.ticker_sym = str(ticker.ticker)
        self.company_name = str(name)
        self.ticker_tags = ticker.tags    
    
    def build_queries(self, ticker=None, prime='name', prime_w=1, tag_w=0.05, include_gtags=True, gtags_w=0.01, include_btags=True, btags_w=0.01):
        
        if ticker is not None:
            self.set_ticker_data(ticker)
        
        if prime=='name':
            self.prime_q = str(self.company_name)
            self.prime_type=1
        elif prime=='ticker':
            self.prime_q = str(self.ticker_sym)
            self.prime_type=2
        elif prime=='both':
            self.prime_q = str(self.company_name)+" "+str(self.ticker_sym)
        
        gtags = ['investing','analysis','analyst','upgrade','downgrade']
        btags = ['sentiment','opinion','outlook']
        
        exp = []
        for t in self.ticker_tags:
            exp.append([t, tag_w])
        if include_gtags:
            for t in gtags:
                exp.append([t, gtags_w])
        if include_btags:
            for t in btags:
                exp.append([t, btags_w])
        
        self.expanded_q=exp
    
    #******************************************************************************
    #------------------------------Sub Dividing-------------------------------------
    #******************************************************************************
    
    def get_pgraphs(self, doc, cutoff, method):
        #updated get_pgraphs() with method for cutoff
        #cut off method:
        #sen: number of sentences
        #word: number of words  

        pgraphs=[]
        freshsoup = re.split('\n\n',doc)
        for x in range(0,len(freshsoup)):
            if method=='word':
                words = len(str(freshsoup[x]).strip().split(' ',maxsplit=cutoff))
                if words>cutoff:
                    pgraphs.append(freshsoup[x])
                    
            elif method=='sen':
                sens = len(re.findall("\.",str(freshsoup[x]).strip()))
                if sens>cutoff:
                    pgraphs.append(freshsoup[x])
                    
        return pgraphs
    
    def split_doc(self, doc):         
        
        if len(doc)>0:
            avgwrds = 15
            estsens = len(doc)/avgwrds
        
            if len(re.findall('\.',doc))>=estsens:
                cut_point = doc.rfind('.', 0, int(len(doc)/2))
                if cut_point<=0:
                    cut_point = int(len(doc)/2)
            else:
                cut_point = int(len(doc)/2)

            d1 = doc[0:cut_point]
            d2 = doc[cut_point+1:]
            
            tkns1 = int(len(self.tokenizer(d1)['input_ids']))

            if tkns1>self.max_tokens:
                self.split_doc(d1)
            else:
                if len(d1)>0:
                    self.subs.append(d1)

            tkns2 = int(len(self.tokenizer(d2)['input_ids']))

            if tkns2>self.max_tokens:
                self.split_doc(d2)
            else:
                if len(d2)>0:
                    self.subs.append(d2)

    
    def get_subdocs(self, pgraphs):
        #Updated get_subdocs with iterative slicing 
        #Changed to recursive slicing
        #ensure sub_docs tokens will not exceed max_tokens for sentiment model
        self.subs=[]

        for x in range(0, len(pgraphs)):
            sen_cnt = len(re.split('\n|\. ',pgraphs[x]))
            tkns = int(len(tokenizer(pgraphs[x])['input_ids']))

            if tkns<self.max_tokens:
                self.subs.append(pgraphs[x])
            else:
                self.split_doc(pgraphs[x])
        
        return self.subs
        
    def sub_divide(self, tokenizer, cutoff=1, method='sen'):

        #creates a dictionary of sub_docs divided from each document in the corpus
        #method: using get_pgraphs() followed by get_subdocs()
        #output form: dict{ document_id : [subdoc_1, subdoc_2 ... subdoc_n] }

        subbed_data = {}
        self.tokenizer=tokenizer
        
        if len(self.pruned_docs)==0:
            self.prune_docs()
        
        for x in range(0, len(self.documents)):
            
            #only include documents that made the first relevance cut
            if x in self.pruned_docs:
                pg = self.get_pgraphs(self.documents[x], cutoff, method)
                subs = self.get_subdocs(pg)
                subbed_data[x]=subs
        
        self.sub_docs = subbed_data
        self.sub_list=[]
        for x in self.sub_docs.keys():
            for y in self.sub_docs[x]:
                self.sub_list.append(y)
        

        
    #******************************************************************************
    #----------------------------------Relevance Scoring---------------------------
    #******************************************************************************  
    
    def rank_docs(self, ranker, min_max=0.02):
        query = self.prime_q
        scr = ranker.score(query, self.documents)
        if np.max(scr)>min_max:
            self.document_scores = scr
        else:
            print('Very low relevance scores, attempting requery')
            #rebuilding queries with prime query being alternate of current prime(name or ticker)
            if self.prime_type==1:
                self.build_queries(prime='ticker')
            else:
                self.build_queries(prime='name')
        
        query = self.prime_q
        scr = ranker.score(query, self.documents)
        if np.max(scr)>min_max:
            self.document_scores = scr
        else:
            print('Scoring on built queries from ticker resulting in unusually low scores')
            print('Change query manually in rank_docs function or adjust min_max')
        
    def rank_subdocs(self, ranker, expanded=False):
        sub_vecs={}
        
        if expanded==False:
            query=self.prime_q
            for x in self.sub_docs.keys():
                sub_vec = ranker.score(query, self.sub_docs[x])
                sub_vecs[x]=sub_vec
        else:
            query = self.expanded_q
            for x in self.sub_docs.keys():
                sub_vec = ranker.score_expanded(query, self.sub_docs[x])
                sub_vecs[x]=sub_vec
            
        self.subdoc_scores = sub_vecs
        
        self.sub_list_scores=[]
        for x in self.subdoc_scores.keys():
            for y in self.subdoc_scores[x]:
                self.sub_list_scores.append(y)
    
    def rank_relevant(self, ranker, expanded=True):
        sub_vecs={}
        
        if expanded==False:
            query=self.prime_q
            for x in self.relevant_set.keys():
                sub_vec = ranker.score(query, self.relevant_set[x])
                sub_vecs[x]=sub_vec
        else:
            query=self.expanded_q
            for x in self.relevant_set.keys():
                sub_vec = ranker.score_expanded(query, self.relevant_set[x])
                sub_vecs[x]=sub_vec
            
        self.relevant_scores = sub_vecs
        
        self.rel_list_scores=[]
        for x in self.relevant_scores.keys():
            for y in self.relevant_scores[x]:
                self.rel_list_scores.append(y)
       
        
    #******************************************************************************
    #----------------------------Pruning Relevant Set------------------------------
    #******************************************************************************
    def prune_docs(self, method='finite', cutoff=0):
        
        '''
        method percentile: cutoff is the percentile to lower bound the document scores on
        method finite: cutoff is a hard value to cutoff scores on
        
        store the indexes of the documents that have scores over the cutoff
        these indexes will be used in the creation of the subdocs
        '''
        
        if method=='percentile':
            p = np.percentile(self.document_scores, cutoff)
            cuts = np.where(self.document_scores<p)
            
            for x in range(0, len(self.documents)):
                if x not in cuts:
                    self.pruned_docs.append(x)
                                                 
        elif method=='finite':
            for x in range(0, len(self.documents)):
                if self.document_scores[x]>cutoff:
                    self.pruned_docs.append(x)
        
        
            
    def prune_subdocs(self, method='finite', cutoff=0):
        
        '''
        method percentile: cutoff is the percentile to lower bound the document scores on
        method finite: cutoff is a hard value to cutoff scores on
        
        '''
        
        if method=='percentile':
            p = np.percentile(self.sub_list_scores, cutoff)
            prune={}
            for x in self.sub_docs.keys():
                for y in range(0, len(sub_docs[x])):
                    if subdoc_scores[x][y]>p:
                        if x not in prune.keys():
                            prune[x]=[y]
                        else:
                            prune[x].append(y)
            self.pruned_subdocs=prune
            
        elif method=='finite':
            prune={}
            for x in self.sub_docs.keys():
                for y in range(0, len(self.sub_docs[x])):
                    if self.subdoc_scores[x][y]>cutoff:
                        if x not in prune.keys():
                            prune[x]=[y]
                        else:
                            prune[x].append(y)
            
            self.pruned_subdocs=prune
        
    def make_relevant(self):
        for x in self.pruned_subdocs.keys():
            self.relevant_set[x]=[]
            self.relevant_scores[x]=[]
            for y in self.pruned_subdocs[x]:
                self.relevant_set[x].append(self.sub_docs[x][y])
                self.relevant_scores[x].append(self.subdoc_scores[x][y])
        
        self.rel_list=[]
        for x in self.relevant_set.keys():
            for y in self.relevant_set[x]:
                self.rel_list.append(y)
        
    def prune_relevant(self, method='percentile',cutoff=15):
        #Relevant is as low as it goes, these will be adjusted directly when pruned
        
        if method=='percentile':
            cut = np.percentile(self.rel_list_scores, cutoff)
        else:
            cut = cutoff
        
        subbed_data = self.relevant_set
        sub_scores = self.relevant_scores
        
        for x in self.relevant_set.keys():

            subbed_data[x] = [xv if c else None for c, xv in zip(sub_scores[x]>cut, subbed_data[x])]
            subbed_data[x] = [y for y in subbed_data[x] if y!=None]
            sub_scores[x] = [y for y in sub_scores[x] if y>cut]
        
        self.relevant_set = {k: v for k, v in subbed_data.items() if len(v) > 0}
        self.relevant_scores={k: v for k, v in sub_scores.items() if len(v) > 0}
        
        self.rel_list=[]
        for x in self.relevant_set.keys():
            for y in self.relevant_set[x]:
                self.rel_list.append(y)
        
 


In [6]:
fullstart = timeit.default_timer()

In [7]:
#pull api keys from the config file
cfig=config()

In [8]:
tick = Ticker(config=cfig, t="MSFT", source='alpha')

In [9]:
#Define Web_query object
wq=web_query(cfig)

In [10]:
start = timeit.default_timer()

#testq=str(tick.name)

d_start="9/1/2021"

#query all of the news apis in web_query object
#wq.query_all(query=tick.name, ticker=tick.ticker, d_start=d_start, threaded=False, pages=1)

#web_query_v2 for ticker object handling and multi-page api requests (when they work)
wq.query_all(tickerobj=tick, d_start=d_start, threaded=True, pages=4)

#compile results into a singular dataframe
wq.compile_results()

end = timeit.default_timer()
print(end-start)

NewsAPI: 100 page: 1
Polygon: 200
Currents: 164 page: 1
Usearch: 50 page: 1
error on Newsapi page: 2
Currents: 0 page: 2
Usearch: 50 page: 2
error on Newsapi page: 3
Currents: 0 page: 3
Usearch: 50 page: 3
error on Newsapi page: 4
Currents: 0 page: 4
Usearch: 50 page: 4
8.405953100000033


In [11]:
print(len(wq.results))

625


In [12]:
start = timeit.default_timer()

#scrap text from the results urls to form documents
wq.scrape_results(threaded=True, max_docs='max')

end = timeit.default_timer()
print('scrape time:',end-start)
print('time per url:', (end-start)/len(wq.results))

IntProgress(value=0, max=625)

scrape time: 53.90686819999996
time per url: 0.08625098911999994


In [13]:
df = wq.get_results()
print(len(df))

625


In [14]:
#build corpus from web query results
corpus=Corpus()
#store the web query data frame in the corpus for referencing urls and titles
corpus.set_results(df)
#assign corpus documents as the web query documents
corpus.set_corpus(wq.documents)

In [15]:
#pull in stop words and build corpus vocabulary 
stopwords=[]
with open('util/stopwords.txt') as f:
    stopwords.append(f.read().splitlines())
stopwords=stopwords[0]

corpus.build_vocabulary(stopwords)

print("Vocabulary size:" + str(len(corpus.vocabulary)))
print("Number of documents:" + str(len(corpus.documents)))

Vocabulary size:49838
Number of documents:547


In [16]:
bm25 = BM25(norm='l2', smooth_idf=True, stopwords=stopwords, sublinear_tf=True)
bm25.fit(corpus.documents)

In [17]:
corpus.build_queries(ticker=tick)
print('Prime query:',corpus.prime_q)
corpus.rank_docs(ranker=bm25)
corpus.prune_docs()

Prime query: Microsoft


In [18]:
#bm25.score(corpus.ticker_sym, corpus.documents)

In [19]:
start = timeit.default_timer()

#create the sub_documents, wrapper to run multiple functions
#passing in the tokenizer to save a little on class dependencies

corpus.sub_divide(tokenizer=tokenizer, cutoff=2, method='sen')
print('Sub-docs:',len(corpus.sub_list))
print('Source count:', len(corpus.sub_docs))

end = timeit.default_timer()
print(end-start)

Token indices sequence length is longer than the specified maximum sequence length for this model (1157 > 512). Running this sequence through the model will result in indexing errors


Sub-docs: 4480
Source count: 450
24.68065989999991


In [20]:
corpus.rank_subdocs(ranker=bm25)

In [21]:
corpus.prune_subdocs()

In [22]:
print("Number of sub_docs:",len(corpus.sub_list))
t = np.array(corpus.sub_list_scores)
z=np.where(t==0.0)
print("Number zero ranked:", len(z[0]))
print("percent of useless subdocs:", round((len(z[0])/len(corpus.sub_list))*100,2),"%")

Number of sub_docs: 4480
Number zero ranked: 3144
percent of useless subdocs: 70.18 %


In [23]:
corpus.make_relevant()
print('relevant sources:',len(corpus.relevant_set))
print('relevant sub_docs:',len(corpus.rel_list))

relevant sources: 431
relevant sub_docs: 1336


In [24]:
corpus.rank_relevant(ranker=bm25)

In [25]:
#corpus.prune_relevant()
print('relevant sources:',len(corpus.relevant_set))
print('relevant sub_docs:',len(corpus.rel_list))

relevant sources: 431
relevant sub_docs: 1336


In [26]:
relevant_set = corpus.relevant_set
relevant_scores = corpus.relevant_scores
print(len(relevant_set))

431


In [27]:
#run the new relevant set through distilled-BERT and get sentiment classifications
start = timeit.default_timer()

sentiments = get_sentiments(relevant_set)

end = timeit.default_timer()
print(end-start)

IntProgress(value=0, max=1336)

350.3034679


In [28]:
lens=[]
for x in relevant_set.keys():
    for y in range(0, len(relevant_set[x])):
        lens.append(len(relevant_set[x][y]))

In [29]:
avlen=np.mean(lens)

In [30]:
len_weight={}
for x in relevant_set.keys():
    l=[]
    for y in range(0, len(relevant_set[x])):
        l.append(len(relevant_set[x][y])/avlen)
    len_weight[x]=l

In [31]:
adjusted_rel = {}
for x in relevant_scores.keys():
    l=[]
    for y in range(0, len(relevant_scores[x])):
        l.append(relevant_scores[x][y] * len_weight[x][y])
    adjusted_rel[x]=l

In [32]:
rel_scores={}
adj_rel_scores={}

for x in relevant_scores.keys():
    rel_scores[x]=np.mean(relevant_scores[x])
    adj_rel_scores[x]=np.mean(adjusted_rel[x])
    

In [33]:
sents={}
for x in sentiments:
    sents[x]=np.mean(sentiments[x])

In [34]:
rw_scores=[]
lrw_scores=[]
for x in relevant_scores.keys():
    for y in range(0, len(relevant_scores[x])):
        rw_scores.append(relevant_scores[x][y] * sentiments[x][y])
        lrw_scores.append(adjusted_rel[x][y] * sentiments[x][y])

In [35]:
scores={}
la_scores={}

for x in relevant_scores.keys():
    rws=0
    lrws=0
    for y in range(0, len(relevant_scores[x])):
        rws += (relevant_scores[x][y] * sentiments[x][y])
        lrws += (adjusted_rel[x][y] * sentiments[x][y])
    scores[x]=rws
    la_scores[x]=lrws

In [36]:
print("Average Relevance weighted Sentiment:", np.mean(rw_scores).round(4))
print("Average Length adjusted Relevance weighted sentiment:", np.mean(lrw_scores).round(4))


Average Relevance weighted Sentiment: 0.1842
Average Length adjusted Relevance weighted sentiment: 0.2301


In [37]:
df.head()

title  \
0  European UCaaS Market Growth Opportunities Rep...   
1  New Report Claims Apple Is In For A Bumper 202...   
2  Kremlin-Backed Hackers Still Stealing U.S. Dat...   
3  PlayStation could launch its own version of Xb...   
4  The Worldwide Cloud Services Industry is Expec...   

                                                 url              pub_date  
0  https://www.businesswire.com/news/home/2021120...  2021-12-06T17:54:08Z  
1  https://www.redmondpie.com/new-report-claims-a...  2021-12-06T17:52:33Z  
2  https://www.newsweek.com/kremlin-backed-hacker...  2021-12-06T17:47:35Z  
3  https://www.independent.co.uk/life-style/gadge...  2021-12-06T17:45:46Z  
4  https://www.prnewswire.com/news-releases/the-w...  2021-12-06T17:45:00Z

In [38]:
#need weight adjusted score per source
newdf = df
newdf['scores'] = pd.Series(scores)
newdf['sentiments'] = pd.Series(sents)
newdf['relevance'] = pd.Series(rel_scores)

In [39]:
newdf

title  \
0   European UCaaS Market Growth Opportunities Rep...   
1   New Report Claims Apple Is In For A Bumper 202...   
2   Kremlin-Backed Hackers Still Stealing U.S. Dat...   
3   PlayStation could launch its own version of Xb...   
4   The Worldwide Cloud Services Industry is Expec...   
..                                                ...   
45  Axiom Investors LLC DE Has $310.23 Million Sta...   
46  TRB Advisors LP Increases Stock Holdings in Mi...   
47  Pinnacle Holdings LLC Sells 69 Shares of Micro...   
48  Ensign Peak Advisors Inc Reduces Holdings in M...   
49  Liberty Wealth Management LLC Increases Stock ...   

                                                  url              pub_date  \
0   https://www.businesswire.com/news/home/2021120...  2021-12-06T17:54:08Z   
1   https://www.redmondpie.com/new-report-claims-a...  2021-12-06T17:52:33Z   
2   https://www.newsweek.com/kremlin-backed-hacker...  2021-12-06T17:47:35Z   
3   https://www.independent.co.uk/life-style/gadge...  2021-12-06T17:45:46Z   
4   https://www.prnewswire.com/news-releases/the-w...  2021-12-06T17:45:00Z   
..                                                ...                   ...   
45  https://www.americanbankingnews.com/2021/10/13...   2021-10-13T08:04:41   
46  https://theenterpriseleader.com/2021/10/12/trb...   2021-10-12T11:06:42   
47  https://www.tickerreport.com/banking-finance/7...   2021-10-12T09:10:41   
48  https://www.tickerreport.com/banking-finance/7...   2021-10-12T09:10:41   
49  https://www.tickerreport.com/banking-finance/7...   2021-10-12T08:38:44   

      scores  sentiments  relevance  
0  -1.099412   -0.542592   0.452360  
1   0.337532    0.999912   0.337562  
2        NaN         NaN        NaN  
3   0.000000    0.999865   0.000000  
4   0.222978    0.964981   0.231069  
..       ...         ...        ...  
45  0.000000    0.012755   0.000000  
46  0.317742    0.329731   0.279079  
47  0.438102   -0.016872   0.226751  
48 -0.548814   -0.015381   0.274675  
49  0.121129    0.518092   0.233799  

[625 rows x 6 columns]

In [40]:
newdf = newdf.dropna()

In [41]:
newdf.head()

title  \
0  European UCaaS Market Growth Opportunities Rep...   
1  New Report Claims Apple Is In For A Bumper 202...   
3  PlayStation could launch its own version of Xb...   
4  The Worldwide Cloud Services Industry is Expec...   
5         Zoom Video Communications: Time To Zoom In   

                                                 url              pub_date  \
0  https://www.businesswire.com/news/home/2021120...  2021-12-06T17:54:08Z   
1  https://www.redmondpie.com/new-report-claims-a...  2021-12-06T17:52:33Z   
3  https://www.independent.co.uk/life-style/gadge...  2021-12-06T17:45:46Z   
4  https://www.prnewswire.com/news-releases/the-w...  2021-12-06T17:45:00Z   
5  https://seekingalpha.com/article/4473639-zoom-...  2021-12-06T17:40:38Z   

     scores  sentiments  relevance  
0 -1.099412   -0.542592   0.452360  
1  0.337532    0.999912   0.337562  
3  0.000000    0.999865   0.000000  
4  0.222978    0.964981   0.231069  
5  0.140195    0.999819   0.070109

In [42]:
#pd.to_datetime(newdf['pub_date'], format='%Y-%m-%dT%H:%M:%S')
newdf.loc[:,'pub_date'] = pd.to_datetime(newdf.loc[:,'pub_date'], infer_datetime_format=True, utc=True)
newdf.loc[:,'pub_date'] = pd.to_datetime(newdf.loc[:,'pub_date'].dt.strftime('%Y/%m/%d'))


C:\Users\rahul\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [43]:
newdf.head()

title  \
0  European UCaaS Market Growth Opportunities Rep...   
1  New Report Claims Apple Is In For A Bumper 202...   
3  PlayStation could launch its own version of Xb...   
4  The Worldwide Cloud Services Industry is Expec...   
5         Zoom Video Communications: Time To Zoom In   

                                                 url   pub_date    scores  \
0  https://www.businesswire.com/news/home/2021120... 2021-12-06 -1.099412   
1  https://www.redmondpie.com/new-report-claims-a... 2021-12-06  0.337532   
3  https://www.independent.co.uk/life-style/gadge... 2021-12-06  0.000000   
4  https://www.prnewswire.com/news-releases/the-w... 2021-12-06  0.222978   
5  https://seekingalpha.com/article/4473639-zoom-... 2021-12-06  0.140195   

   sentiments  relevance  
0   -0.542592   0.452360  
1    0.999912   0.337562  
3    0.999865   0.000000  
4    0.964981   0.231069  
5    0.999819   0.070109

In [44]:
plotdf = newdf[['pub_date','sentiments','relevance','scores']]
plotdf.head()

pub_date  sentiments  relevance    scores
0 2021-12-06   -0.542592   0.452360 -1.099412
1 2021-12-06    0.999912   0.337562  0.337532
3 2021-12-06    0.999865   0.000000  0.000000
4 2021-12-06    0.964981   0.231069  0.222978
5 2021-12-06    0.999819   0.070109  0.140195

In [45]:
print(np.min(plotdf['pub_date']))
print(np.max(plotdf['pub_date']))

2021-09-01 00:00:00
2021-12-06 00:00:00


In [46]:
pt_sn = plotdf.groupby(plotdf['pub_date'])['sentiments'].mean()
pt_rl = plotdf.groupby(plotdf['pub_date'])['relevance'].mean()
pt_sc = plotdf.groupby(plotdf['pub_date'])['scores'].mean()
pt_dt = pd.DataFrame()
pt_dt = plotdf['sentiments'].groupby(plotdf['pub_date']).count()

In [47]:
pt_sn = pt_sn.to_frame()
pt_rl = pt_rl.to_frame()
pt_sc = pt_sc.to_frame()
pt_dt = pt_dt.to_frame().rename(columns={"sentiments":"Volume"})

In [48]:
dff = pd.concat([pt_rl, pt_sn, pt_sc, pt_dt], axis=1)


In [49]:
dff = dff.reset_index()
dff = dff.drop([0,1])
dff = dff.set_index('pub_date')


In [50]:
drange='1mo'
interval='1d'

yheaders = {'x-api-key':cfig.yahoo}
url = ('https://yfapi.net/v8/finance/spark?'
        'interval='+interval+
        '&range='+drange+
        '&symbols='+tick.ticker)

response = requests.request("GET",url,headers=yheaders)
if response.status_code==200:
        response = response.json()



In [51]:
date = response[tick.ticker]['timestamp']
price = response[tick.ticker]['close']
df2 = pd.DataFrame()
df2['date']=date
df2.loc[:,'date'] = pd.to_datetime(df2.loc[:,'date'], unit='s')
df2.loc[:,'date'] = pd.to_datetime(df2.loc[:,'date'].dt.strftime('%Y/%m/%d'))
df2['price']=price
df2 = df2.set_index('date')


In [52]:
tmp = dff[dff.columns[0:3]]
tmp.head()

relevance  sentiments    scores
pub_date                                   
2021-09-07   0.088146   -0.007058 -0.087626
2021-09-08   0.141232    0.999199  0.141205
2021-09-09   0.549150    0.604001  0.871174
2021-09-13   0.470764    0.491024  0.199041
2021-09-14   0.169245    0.971007  0.675990

In [53]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=3)

dff.plot(ax=axes[0], figsize=(20,20))
df2.plot(ax=axes[1], figsize=(20,20))
tmp.plot(ax=axes[2], figsize=(20,20))

<AxesSubplot:xlabel='pub_date'>

In [54]:
drange='6mo'
interval='1d'


url=('https://yfapi.net/v8/finance/chart/'+tick.ticker+"?"+
    'comparisons='+tick.ticker+
    '&range='+drange+
    '&region=US'
    '&interval='+interval+
    '&lang=en')
response = requests.request("GET",url,headers=yheaders)
if response.status_code==200:
        response = response.json()            

In [55]:
date = response['chart']['result'][0]['timestamp']
volume = response['chart']['result'][0]['indicators']['quote'][0]['volume']
high = response['chart']['result'][0]['indicators']['quote'][0]['high']
low = response['chart']['result'][0]['indicators']['quote'][0]['low']
opn = response['chart']['result'][0]['indicators']['quote'][0]['open']
cls = response['chart']['result'][0]['indicators']['quote'][0]['close']

In [56]:
df3 = pd.DataFrame()
df3['date']=date
df3['high']=high
df3['low']=low
df3['open']=opn
df3['close']=cls
df3['volume']=volume
df3.loc[:,'date'] = pd.to_datetime(df3.loc[:,'date'], unit='s')
df3.loc[:,'date'] = pd.to_datetime(df3.loc[:,'date'].dt.strftime('%Y/%m/%d'))

In [57]:
dff=dff.reset_index()

In [128]:
app = dash.Dash()

def make_stock_chart():
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,vertical_spacing=0.02, subplot_titles=(tick.ticker,'Sentiment_Score', 'Used News Volume'), 
               row_width=[0.4, 0.4, 1.5], specs=[[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}]])
    fig.add_trace(go.Candlestick(x=df3['date'],
                open=df3['open'], high=df3['high'],
                low=df3['low'], close=df3['close'], name="Price"),
               secondary_y=True, row=1,col=1)

    fig.add_trace(go.Bar(x=df3['date'], y=df3['volume'],opacity=0.2, name='Volume'),secondary_y=False)
    fig.add_trace(go.Line(x=dff['pub_date'], y=dff['scores'], showlegend=False), row=2, col=1)
    fig.add_trace(go.Bar(x=dff['pub_date'],y=dff['Volume'],opacity=0.2, name='Used News Volume'),row=3, col=1)
    fig.update_layout(xaxis_rangeslider_visible=False, width=1000, height=750)
    fig.layout.yaxis2.showgrid=False
    return fig

app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Financial News Sentiment Analysis of ' + tick.ticker, style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),

        
        dcc.Graph(id = 'line_plot', figure = make_stock_chart())    
    ])

app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2021 18:14:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2021 18:14:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2021 18:14:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2021 18:14:18] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2021 18:14:18] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2021 18:14:18] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -


In [ ]:
fullstop = timeit.default_timer()


In [ ]:
print('Tiny Report')
print('Full compile, sub-divide, analyze and graph time:', round((fullstop-fullstart)/60,2), "minutes")
print('Urls scraped:', len(corpus.documents))
print('Relevant Subdocs created:', len(corpus.rel_list))